In [ ]:
from tarea1.algoritmos import hola

hola() 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# from matplotlib.pyplot import rcParams
# %matplotlib inline
# rcParams['font.family'] = 'serif'
# rcParams['font.size'] = 16

20:56 empieza a hablar de un ejercicio.

Obtener las derivadas para el método de gradiente.
Hay que armar el vector gradiente.

 
Sympy: Programación simbólica. Ecuaciones como objetos, puede ser complicado.

**Mejor hacer el gradiente a mano.**


# Gradiente con Backtracking Line Search 24:50

Alpha que es multiplicador de un compadre. Entre 0 y 0.5

Beta que actualiza tiempos. Entre 0 y 1 sin tocarlos. 

¿Tiempos o Steps? Tiempos. Siempre se refiere de esa manera.

Hay que crear una igualdad, se evalua. Siempre que se cumpla se actualiza el tiempo.

Se parte con el punto inicial 0 0 0

---
Delta X es - grad F. (26:24 )


Mientras no se cumpla la igualdad, se actualizará el punto inicial.
P0: (0,0,0)


## 28:43

Hay dos ciclos aniddados

1. Busqueda de la condición. La búsqueda del valor de t. T sub optimo para un sub problema
2. Con ese t, se actualiza el punto.

$$
P^{k+1} = P^K +t \nabla x
$$


## Criterio de parada
Hay varias, el ayudante ocupa una en que la norma del vector gradiente sea aproximadamente 0.
Osea, menor a un umbral.

$$
|| \nabla f(x) || \approx 0
$$


In [ ]:
from dataclasses import dataclass
from symbol import parameters
from typing import Any, Callable
#from numpy.typing import *


@dataclass
class State:
  punto: Any
  alpha: float
  beta: float
  f: Callable[..., Any]
  fd: Callable[..., Any]


@dataclass
class Capture:
  k: int
  FO_optima: list


def state_update(state: State):
  punto = state.punto
  f = state.f
  fd = state.fd

  alpha = state.alpha
  beta = state.beta


  t: float = 1.0

  while True:
    LR = f(punto) + alpha*t * np.dot(np.transpose(fd(punto)), - fd(punto))
    LL = f(punto - t*fd(punto))
    t = t*beta

    if LR>LL:
      punto = punto - t*fd(punto)
      t = 1
      break


def capture_update(state: State, capture: Capture) -> Capture:

  punto = state.punto
  f = state.f

  capture.k += 1
  capture.FO_optima.append(f(punto))


def metodo_gradiente(state: State, epsilon=0.001):
  
  capture = Capture(k=0, FO_optima=list())

  f = state.f
  fd = state.fd

  while True:
    state_update(state)

    if np.abs(np.linalg.norm(fd(state.punto))) <= epsilon:
      break

    capture_update(state, capture)


  capture_update(state, capture)
  print(f(state.punto), capture.k)
  
  return capture


def f(X):
  x0 = X[0]
  x1 = X[1]
  x2 = X[2]
  return np.exp( -x0 + 2*x1 ) + np.exp( 3*x2 - 2*x0 ) + x1**2


def fd(X):
  x0 = X[0]
  x1 = X[1]
  x2 = X[2]

  return np.array([
    -np.exp(-x0 + 2*x1) - 2*np.exp( 3*x2 -2*x0 ),
    2*np.exp(-x0 + 2*x1) + 2*x1,
    3*np.exp(3*x2 - 2*x0)
  ])


initial_state = State(
  np.array([0,0,0]),
  alpha=0.3,
  beta=0.6,
  f=f,
  fd=fd
)

results: Capture = metodo_gradiente(initial_state)

In [ ]:

alpha = 0.3
beta = 0.6


FO_optima, k = results

fig, (ax1) = plt.subplots(nrows=1, figsize=(8,5))

time=0
label= rf'$\alpha = {alpha}$, $\beta = {beta}$, time$= {time}$'

ax1.scatter(range(k), FO_optima, s=0.7, label=label)
ax1.set_xlabel("Número de iteraciones")
ax1.set_ylabel("Valor de FO en iteración $k$")
ax1.set_title("Convergencia del método")
ax1.legend(loc="best")
